# Module 0: Environment Setup

This notebook guides you through setting up your environment for the Claude Agent SDK and Amazon Bedrock AgentCore workshop.

## Learning Objectives

By the end of this module, you will:
- Configure AWS credentials and region
- Verify Bedrock model access
- Create S3 bucket and IAM role
- Upload pre-generated demo data to S3
- Set up Amazon Athena database and tables
- Validate your environment is ready for the workshop

## Prerequisites

- AWS Account with Bedrock access enabled
- AWS CLI configured with appropriate credentials
- Python 3.14 or higher

## Step 1: Install Dependencies

First, let's install the required Python packages.

In [ ]:
# Install workshop dependencies
!pip install -e .. --quiet
print("Dependencies installed successfully!")

## Step 2: Configure Environment Variables

The `.env` file will be automatically created and populated with your AWS configuration.

| Variable | Description | How It's Set |
|----------|-------------|--------------|
| `AWS_REGION` | AWS region for services | Auto-detected from boto3 session |
| `AWS_ACCOUNT_ID` | Your 12-digit AWS account ID | Auto-detected via STS |
| `ATHENA_DATABASE` | Database name for student analytics | Fixed: `student_analytics` |
| `ATHENA_OUTPUT_LOCATION` | S3 path for Athena query results | Auto-generated |
| `S3_BUCKET_NAME` | S3 bucket for data storage | `student-analytics-agent-<account_id>` |

In [ ]:
import os
import boto3
from pathlib import Path

workshop_root = Path("..")
env_file = workshop_root / ".env"
env_example = workshop_root / ".env.example"

# Auto-detect AWS configuration
print("Detecting AWS configuration...")

# Get region from boto3 session
session = boto3.Session()
aws_region = session.region_name or 'us-east-1'
print(f"  AWS Region: {aws_region}")

# Get account ID from STS
sts = boto3.client('sts', region_name=aws_region)
account_id = sts.get_caller_identity()['Account']
print(f"  AWS Account ID: {account_id}")

# Generate bucket name
bucket_name = f"student-analytics-agent-{account_id}"
print(f"  S3 Bucket Name: {bucket_name}")

# Create .env content with all required configurations
env_content = f"""# AWS Configuration
AWS_REGION={aws_region}
AWS_ACCOUNT_ID={account_id}

# Claude Agent SDK - Use Bedrock as the model provider
CLAUDE_CODE_USE_BEDROCK=1

# Athena Configuration
ATHENA_DATABASE=student_analytics
ATHENA_OUTPUT_LOCATION=s3://{bucket_name}/athena-results/

# S3 Bucket for data and results
S3_BUCKET_NAME={bucket_name}

# Bedrock Model Configuration
ANTHROPIC_MODEL=global.anthropic.claude-opus-4-5-20251101-v1:0
ANTHROPIC_SMALL_FAST_MODEL=global.anthropic.claude-haiku-4-5-20251001-v1:0
CLAUDE_CODE_MAX_OUTPUT_TOKENS=64000
MAX_THINKING_TOKENS=4800
DISABLE_PROMPT_CACHING=1

# AgentCore Configuration (populated after deployment)
# AGENTCORE_AGENT_ID=
# AGENTCORE_AGENT_ARN=
"""

# Write .env file
with open(env_file, 'w') as f:
    f.write(env_content)

print(f"\n✅ Created {env_file} with auto-detected configuration!")

In [ ]:
# Load and verify environment variables
from dotenv import load_dotenv

# Use override=True to ensure fresh values are loaded
load_dotenv(env_file, override=True)

# Display configuration
print("Current Configuration:")
print(f"  AWS_REGION: {os.getenv('AWS_REGION')}")
print(f"  AWS_ACCOUNT_ID: {os.getenv('AWS_ACCOUNT_ID')}")
print(f"  ATHENA_DATABASE: {os.getenv('ATHENA_DATABASE')}")
print(f"  S3_BUCKET_NAME: {os.getenv('S3_BUCKET_NAME')}")
print(f"  ATHENA_OUTPUT_LOCATION: {os.getenv('ATHENA_OUTPUT_LOCATION')}")

print("\n✅ Environment variables loaded successfully!")

## Step 3: Verify AWS Credentials

Let's verify your AWS credentials are properly configured.

In [ ]:
import boto3

try:
    # Verify AWS credentials
    sts = boto3.client('sts', region_name=os.getenv('AWS_REGION', 'us-east-1'))
    identity = sts.get_caller_identity()
    
    print("✅ AWS credentials verified!")
    print(f"   Account: {identity['Account']}")
    print(f"   User/Role: {identity['Arn'].split('/')[-1]}")
except Exception as e:
    print(f"❌ AWS credentials error: {e}")
    print("\nPlease configure AWS credentials:")
    print("  aws configure")
    print("  or set AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY environment variables")

## Step 4: Verify Bedrock Model Access

We need access to Claude models through Amazon Bedrock.

In [ ]:
import boto3

try:
    # Check Bedrock model availability
    bedrock = boto3.client('bedrock', region_name=os.getenv('AWS_REGION', 'us-east-1'))
    
    # List foundation models
    response = bedrock.list_foundation_models(byProvider='Anthropic')
    claude_models = [m for m in response['modelSummaries'] if 'claude' in m['modelId'].lower()]
    
    print("✅ Bedrock access verified!")
    print(f"\nAvailable Claude models ({len(claude_models)}):")
    for model in claude_models[:5]:  # Show first 5
        print(f"   - {model['modelId']}")
    if len(claude_models) > 5:
        print(f"   ... and {len(claude_models) - 5} more")
        
except Exception as e:
    print(f"❌ Bedrock access error: {e}")
    print("\nPlease ensure:")
    print("  1. Bedrock is enabled in your region")
    print("  2. Claude models are enabled in Bedrock console")
    print("  3. Your IAM role has bedrock:ListFoundationModels permission")

## Step 5: Create S3 Bucket (if needed)

Create an S3 bucket to store Athena query results and demo data.

In [ ]:
import boto3

bucket_name = os.getenv('S3_BUCKET_NAME')
region = os.getenv('AWS_REGION', 'us-east-1')

if not bucket_name or bucket_name.startswith('YOUR_'):
    print("⚠️  Please set S3_BUCKET_NAME in .env file")
else:
    s3 = boto3.client('s3', region_name=region)
    
    try:
        # Check if bucket exists
        s3.head_bucket(Bucket=bucket_name)
        print(f"✅ S3 bucket '{bucket_name}' exists and is accessible!")
    except s3.exceptions.ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == '404':
            print(f"Creating S3 bucket: {bucket_name}")
            try:
                if region == 'us-east-1':
                    s3.create_bucket(Bucket=bucket_name)
                else:
                    s3.create_bucket(
                        Bucket=bucket_name,
                        CreateBucketConfiguration={'LocationConstraint': region}
                    )
                print(f"✅ Created S3 bucket: {bucket_name}")
            except Exception as create_error:
                print(f"❌ Failed to create bucket: {create_error}")
        else:
            print(f"❌ S3 bucket error: {e}")

## Step 5b: Create IAM Role for AgentCore

Create the IAM role that AgentCore will use to access Bedrock, Athena, S3, and CloudWatch.

This role includes:
- **Athena/Glue**: Query execution and catalog access
- **S3**: Read/write for data and query results
- **Bedrock**: Model invocation for Claude
- **CloudWatch Logs**: Basic logging + delivery permissions for observability
- **ECR**: Container image access for runtime

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError

iam = boto3.client('iam')
region = os.getenv('AWS_REGION', 'us-east-1')
bucket_name = os.getenv('S3_BUCKET_NAME')
account_id = os.getenv('AWS_ACCOUNT_ID', '')

# If account_id not set, get it from STS
if not account_id:
    sts = boto3.client('sts')
    account_id = sts.get_caller_identity()['Account']

role_name = "StudentAnalyticsAgentCoreRole"

# Trust policy - allows AgentCore, Bedrock, Lambda, and ECS to assume this role
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": [
                    "lambda.amazonaws.com",
                    "bedrock.amazonaws.com",
                    "bedrock-agentcore.amazonaws.com",
                    "ecs-tasks.amazonaws.com"
                ]
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Permissions policy for the agent
permissions_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AthenaQueryExecution",
            "Effect": "Allow",
            "Action": [
                "athena:StartQueryExecution",
                "athena:GetQueryExecution",
                "athena:GetQueryResults",
                "athena:StopQueryExecution",
                "athena:GetWorkGroup",
                "athena:GetDataCatalog",
                "athena:GetDatabase",
                "athena:GetTableMetadata",
                "athena:ListQueryExecutions"
            ],
            "Resource": [
                "arn:aws:athena:*:*:workgroup/*",
                "arn:aws:athena:*:*:datacatalog/*"
            ]
        },
        {
            "Sid": "GlueCatalogAccess",
            "Effect": "Allow",
            "Action": [
                "glue:GetDatabase",
                "glue:GetTable",
                "glue:GetPartitions",
                "glue:GetDatabases",
                "glue:GetTables"
            ],
            "Resource": [
                "arn:aws:glue:*:*:catalog",
                "arn:aws:glue:*:*:database/student_analytics",
                "arn:aws:glue:*:*:table/student_analytics/*"
            ]
        },
        {
            "Sid": "S3AthenaAccess",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket",
                "s3:GetBucketLocation",
                "s3:PutObject",
                "s3:DeleteObject"
            ],
            "Resource": [
                f"arn:aws:s3:::{bucket_name}",
                f"arn:aws:s3:::{bucket_name}/*"
            ]
        },
        {
            "Sid": "BedrockFullAccess",
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:InvokeModelWithResponseStream",
                "bedrock:GetFoundationModel",
                "bedrock:ListFoundationModels",
                "bedrock:GetInferenceProfile",
                "bedrock:ListInferenceProfiles",
                "bedrock:GetAgent",
                "bedrock:InvokeAgent"
            ],
            "Resource": "*"
        },
        {
            "Sid": "AWSMarketplaceAccess",
            "Effect": "Allow",
            "Action": [
                "aws-marketplace:ViewSubscriptions",
                "aws-marketplace:Subscribe",
                "aws-marketplace:Unsubscribe",
                "aws-marketplace:GetEntitlements"
            ],
            "Resource": "*"
        },
        {
            "Sid": "CloudWatchLogsFullAccess",
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents",
                "logs:DescribeLogGroups",
                "logs:DescribeLogStreams",
                "logs:GetLogEvents",
                "logs:FilterLogEvents",
                "logs:GetDelivery",
                "logs:GetDeliverySource",
                "logs:PutDeliveryDestination",
                "logs:GetDeliveryDestinationPolicy",
                "logs:DeleteDeliverySource",
                "logs:PutDeliveryDestinationPolicy",
                "logs:CreateDelivery",
                "logs:GetDeliveryDestination",
                "logs:PutDeliverySource",
                "logs:DeleteDeliveryDestination",
                "logs:DeleteDeliveryDestinationPolicy",
                "logs:DeleteDelivery",
                "logs:UpdateDeliveryConfiguration",
                "logs:DescribeDeliveryDestinations",
                "logs:DescribeDeliverySources",
                "logs:DescribeDeliveries",
                "logs:DescribeConfigurationTemplates",
                "logs:PutResourcePolicy",
                "logs:DescribeResourcePolicies",
                "logs:DeleteResourcePolicy"
            ],
            "Resource": "*"
        },
        {
            "Sid": "XRayTracesAccess",
            "Effect": "Allow",
            "Action": [
                "xray:PutTraceSegments",
                "xray:PutTelemetryRecords",
                "xray:GetSamplingRules",
                "xray:GetSamplingTargets",
                "xray:GetSamplingStatisticSummaries",
                "xray:UpdateTraceSegmentDestination",
                "xray:GetTraceSegmentDestination",
                "xray:UpdateIndexingRule",
                "xray:GetIndexingRules"
            ],
            "Resource": "*"
        },
        {
            "Sid": "ECRImageAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:GetAuthorizationToken",
                "ecr:BatchGetImage",
                "ecr:GetDownloadUrlForLayer",
                "ecr:BatchCheckLayerAvailability"
            ],
            "Resource": "*"
        }
    ]
}

# Check if role already exists
try:
    iam.get_role(RoleName=role_name)
    print(f"✅ IAM role '{role_name}' already exists!")
    
    # Update the policy to ensure it has latest permissions
    print("   Updating permissions policy...")
    iam.put_role_policy(
        RoleName=role_name,
        PolicyName="StudentAnalyticsAgentCorePolicy",
        PolicyDocument=json.dumps(permissions_policy)
    )
    print(f"   ✅ Policy updated with:")
    print(f"      - S3 bucket: {bucket_name}")
    print(f"      - AWS Marketplace access (for Bedrock model subscriptions)")
    print(f"      - X-Ray access (for traces delivery)")
    print(f"      - CloudWatch Logs full access (for observability)")
    
except ClientError as e:
    if e.response['Error']['Code'] == 'NoSuchEntity':
        print(f"Creating IAM role: {role_name}")
        
        # Create the role
        iam.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description="IAM role for Student Analytics Agent on AgentCore Runtime"
        )
        print(f"   ✅ Role created")
        
        # Attach the permissions policy
        iam.put_role_policy(
            RoleName=role_name,
            PolicyName="StudentAnalyticsAgentCorePolicy",
            PolicyDocument=json.dumps(permissions_policy)
        )
        print(f"   ✅ Policy attached with full permissions")
        
        print(f"\n✅ IAM role '{role_name}' created successfully!")
        print(f"   ARN: arn:aws:iam::{account_id}:role/{role_name}")
    else:
        print(f"❌ Error checking/creating IAM role: {e}")

## Step 6: Verify Demo Data

The demo data has been pre-generated and included in the `data/demo_data/` folder. Let's verify it exists.

In [ ]:
# Verify demo data exists
data_dir = workshop_root / "data" / "demo_data"

if data_dir.exists():
    csv_files = list(data_dir.glob("*.csv"))
    if csv_files:
        print(f"✅ Demo data found ({len(csv_files)} files)")
        print("\nData files:")
        for f in sorted(csv_files):
            size_mb = f.stat().st_size / (1024 * 1024)
            print(f"   - {f.name} ({size_mb:.2f} MB)")
    else:
        print("❌ No CSV files found in data/demo_data/")
        print("Please ensure demo data CSV files are in the data/demo_data/ folder.")
else:
    print("❌ Demo data directory not found!")
    print(f"Expected location: {data_dir}")
    print("\nPlease create the data/demo_data/ folder and add the CSV files.")

## Step 7: Set Up Athena Database and Tables

Create the Athena database and external tables pointing to our S3 data.

In [ ]:
import sys
import importlib
sys.path.insert(0, str(workshop_root))

print("Setting up Athena database and tables...")
print("This will:")
print("  1. Create the 'student_analytics' database")
print("  2. Create external tables pointing to S3 data")
print("  3. Verify table accessibility")

proceed = True

if proceed:
    # Import and reload to pick up any changes
    import scripts.setup_athena as setup_athena_module
    importlib.reload(setup_athena_module)
    
    # Call setup_athena with environment variables (already loaded)
    # The function reads S3_BUCKET_NAME and AWS_REGION from env if not provided
    success = setup_athena_module.setup_athena(
        database=str(os.getenv('ATHENA_DATABASE')),
        data_dir=str(workshop_root / "data" / "demo_data")
    )
    
    if success:
        print("\n✅ Athena setup complete!")
    else:
        print("\n❌ Athena setup failed. Check error messages above.")
else:
    print("Skipped Athena setup.")

## Step 8: Verify Athena Query Access

Test that we can execute queries against the student analytics database.

In [ ]:
import sys
sys.path.insert(0, str(workshop_root))

from tools.athena_tools import AthenaQueryExecutor

# Initialize Athena tools
athena = AthenaQueryExecutor(
    database=os.getenv('ATHENA_DATABASE', 'student_analytics'),
    output_location=os.getenv('ATHENA_OUTPUT_LOCATION'),
    results_dir=str(workshop_root / 'results' / 'raw'),
    region=os.getenv('AWS_REGION', 'us-east-1')
)

# Test query
print("Running test query...")
try:
    result = athena.execute_and_download(
        query="SELECT COUNT(*) as total FROM student_enrollment_analytics LIMIT 1",
        local_filename="test_query.csv"
    )
    print(f"\n✅ Athena query successful!")
    print(f"   Results saved to: {result['local_file']}")
    print(f"   Execution time: {result['execution_time_ms']}ms")
    
    # Read and display result
    import pandas as pd
    df = pd.read_csv(result['local_file'])
    print(f"   Total records: {df['total'].iloc[0]:,}")
except Exception as e:
    print(f"❌ Athena query failed: {e}")

## Step 9: Environment Validation Summary

Let's verify all components are properly configured.

In [ ]:
import boto3

print("=" * 60)
print("ENVIRONMENT VALIDATION SUMMARY")
print("=" * 60)

# Check if IAM role exists
iam_role_exists = False
try:
    iam_client = boto3.client('iam')
    iam_client.get_role(RoleName="StudentAnalyticsAgentCoreRole")
    iam_role_exists = True
except:
    pass

checks = [
    ("Python version >= 3.10", sys.version_info >= (3, 14)),
    (".env file exists", env_file.exists()),
    ("AWS credentials configured", 'identity' in dir()),
    ("IAM role created (StudentAnalyticsAgentCoreRole)", iam_role_exists),
    ("Demo data exists", (workshop_root / "data" / "demo_data").exists()),
    ("Skills files present", (workshop_root / ".claude" / "skills").exists()),
    ("CLAUDE.md exists", (workshop_root / "CLAUDE.md").exists()),
]

all_passed = True
for check_name, passed in checks:
    status = "✅" if passed else "❌"
    print(f"  {status} {check_name}")
    if not passed:
        all_passed = False

print("\n" + "=" * 60)
if all_passed:
    print("✅ All checks passed! You're ready for Module 1.")
else:
    print("⚠️  Some checks failed. Please review the issues above.")
print("=" * 60)

## Next Steps

Congratulations! Your environment is set up. Continue to:

**[Module 1a: Basic Agent (No Skills)](../module-1-local-agent/1a-basic-agent-no-skills.ipynb)**

In the next module, you will:
- Run a basic Claude agent WITHOUT skills or context
- See how the agent struggles with database queries
- Understand why skills and context are important

---

*Workshop: Build Agentic AI Applications with Claude Agent SDK and Amazon Bedrock AgentCore*